# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import t
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.weightstats import DescrStatsW, CompareMeans, ttest_ind, ztest, zconfint
from math import sqrt

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head(3)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [6]:
data.shape

(4870, 65)

In [31]:
num_white_resume=len(data[data['race']=='w'])
num_black_resume=len(data[data['race']=='b'])
data['race'].value_counts()

b    2435
w    2435
Name: race, dtype: int64

In [8]:
data['call'].value_counts()

0.0    4478
1.0     392
Name: call, dtype: int64

In [30]:
callw = sum(data[data['race']=='w'].call)
callb = sum(data[data['race']=='b'].call)
not_called_whites = len(data[data['race'] == 'w']) - callw
not_called_blacks = len(data[data['race'] == 'b']) - callb
print ('# of callbacks for white people: ', callw)
print ('# of callbacks for black people: ', callb)
print ('# of not_callbacks for white people: ', not_called_whites)
print ('# of not_callbacks for black people: ', not_called_blacks)


# of callbacks for white people:  235.0
# of callbacks for black people:  157.0
# of not_callbacks for white people:  2200.0
# of not_callbacks for black people:  2278.0


In [39]:
pool= round((callw+callb)/(num_white_resume+num_black_resume),2)
print ('np #1: ', callw * pool)
print ('n(1-p) #1: ', callw * (1-pool))
print ('np #2: ', callb * pool)
print ('n(1-p) #2: ', callb * (1-pool))

np #1:  18.8
n(1-p) #1:  216.2
np #2:  12.56
n(1-p) #2:  144.44


## 1. What test is appropriate for this problem? Does CLT apply?

The appropriate test for this problem is hypothesis test to compare two independent samples. 

The sample size is large to CLT can be applied. Moreover, CLT follows the 10% rule: The sample size must not be bigger than 10% of the entire population. Total number of whites and blacks are around 2000, which is much less than the 10% of the entire population. Hence CLT applies. 

Sample size should be large enough that **np>=10**  and **n(1-p)>=10**, i.e. success and failure numbers should both be greater than 10. According to calculations above, they are always greater than 10. 

## 2. What are the null and alternate hypotheses?

H_0 = There is no difference between call-back rates on resumes from black vs. white

H_A = There is a difference between callback rates on resumes from black vs. white

In [10]:
black = data[data['race'] == 'b']
white = data[data['race'] == 'w']

In [11]:
black.head(2)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [12]:
white.head(2)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [13]:
ind_t_test=stats.ttest_ind(black['call'], white['call'], axis=0)
print('The results of the independent t-test are: \n\tt-value = {:4.3f}\n\tp-value = {:4.6f}'.format(ind_t_test[0],ind_t_test[1]))

The results of the independent t-test are: 
	t-value = -4.115
	p-value = 0.000039


In [14]:
print(ttest_ind(black['call'], white['call'], usevar='unequal'))

(-4.1147052465936262, 3.9429422654060233e-05, 4711.6024487222085)


Using both the StatsModels and Scipy.stats libraries, the p-value is less than 0.05, and the null hypothesis is rejected. There is indeed a difference between the number of calls received from the employer.

Conclusion: There is a significant difference in call-back rates for Resumes.

## 3. Compute margin of error, confidence interval, and p-value

In [15]:
mean_black = black['call'].mean()
print ('mean of callbacks for blacks: ',  mean_black)
mean_white = white['call'].mean()
print ('mean of callbacks for whites: ',  mean_white)
diff_mean = mean_black - mean_white
print('difference of mean of callbacks: ', diff_mean)

mean of callbacks for blacks:  0.0644763857126236
mean of callbacks for whites:  0.09650924056768417
difference of mean of callbacks:  -0.03203285485506058


In [19]:
black['call'].head(4)

2    0.0
3    0.0
7    0.0
8    0.0
Name: call, dtype: float32

Calculate the margin of error and the mean difference between two independent groups with 95% confidence interval.

In [22]:
N1 = 2435
N2 = 2435
df = (N1 + N2 - 2)
std1 = black['call'].std()
std2 = white['call'].std()
std_N1N2 = sqrt( ((N1 - 1)*(std1)**2 + (N2 - 1)*(std2)**2) / df) 

diff_mean = mean_white - mean_black
MoE = t.ppf(0.975, df) * std_N1N2 * sqrt(1/N1 + 1/N2) # because 2.5 - 97.5% corresponds to 95% of the data
print('\n Margin of Error: ', MoE)
print ('\n The difference between groups is {:3.4f} with 95% CI of [{:3.4f} to {:3.4f}]'.format(diff_mean, diff_mean - MoE, diff_mean + MoE))



 Margin of Error:  0.01526193185

 The difference between groups is 0.0320 with 95% CI of [0.0168 to 0.0473]


## 4. Write a story describing the statistical significance in the context or the original problem.

The value of the p-value being less than 0.05 indicates that we can reject the Null hypothesis. At 95% CI, callback rate to black applicants is lower. 